In [ ]:
!pip install -U pip setuptools wheel
!pip install -U 'spacy[cuda12x]'

In [ ]:
!python -m spacy download uk_core_news_lg

In [3]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from google.colab import userdata

# Use a service account.
cred = credentials.Certificate(userdata.get('firebaseCredentialsPath'))

app = firebase_admin.initialize_app(cred)

db = firestore.client()

In [ ]:
from google.cloud.firestore_v1.field_path import FieldPath
from google.cloud.firestore_v1.base_query import FieldFilter
import json

PAGE_SIZE = 100

docs_ref = db.collection("raw-documents")
query = docs_ref.order_by(FieldPath.document_id()).where(filter=FieldFilter("collections", "array_contains_any", ["Бакалаврські роботи", "Магістерські роботи"])).limit(PAGE_SIZE)

i = 0

result = []

while(True):

  docs = list(query.stream())

  if (len(docs) == 0): break;

  last_doc = docs[-1]
  last_id = last_doc.id

  for doc in docs:
    docDict = doc.to_dict()

    if docDict.get("chapters"):
      chapters = docDict.get("chapters")

      for key in chapters.keys():
        result.append(chapters[key])


  query = docs_ref.order_by(FieldPath.document_id()).where(filter=FieldFilter("collections", "array_contains_any", ["Бакалаврські роботи", "Магістерські роботи"])).start_after({FieldPath.document_id(): last_id}).limit(PAGE_SIZE)

  print(f"{i} - {last_id}")
  i = i + 1

with open("/content/drive/MyDrive/diploma-llm/data/lda/raw_dataset.json", "w") as outfile:
  outfile.write(json.dumps(result, indent=4))

In [38]:
with open("/content/drive/MyDrive/diploma-llm/data/lda/raw_dataset.json", "r") as source:
  raw_dataset = json.load(source)

In [ ]:
import spacy
from spacy.tokens import Doc

spacy.require_gpu()
nlp = spacy.load("uk_core_news_lg", enable=["tok2vec", "morphologizer"])

def filter_and_map(doc):

  result = []

  for index, token in enumerate(doc):
    if (token.pos_  in ('NOUN', 'ADJ', 'VERB')):
      result.append(token.text.lower())

  return " ".join(word for word in result)

result = []
processed_counter = 0

for doc in raw_dataset:
  result.append(filter_and_map(nlp(doc)))
  processed_counter += 1
  if processed_counter % 100 == 0:
    print(processed_counter)

# result = [filter_and_map(nlp(doc)) for doc in raw_dataset]


with open("/content/drive/MyDrive/diploma-llm/data/lda/filtered_dataset.json", "w") as outfile:
  outfile.write(json.dumps(result, indent=4))

In [42]:
with open("/content/drive/MyDrive/diploma-llm/data/lda/filtered_dataset.json", "r") as source:
  filtered_dataset = json.load(source)

In [46]:
import spacy
from spacy.tokens import Doc

spacy.require_gpu()
nlp = spacy.load("uk_core_news_lg", enable=["tok2vec", "lemmatizer"])

def map_and_join(doc):

  return " ".join(token.lemma_.lower() for token in doc)

result = []
processed_counter = 0

for doc in filtered_dataset:
  result.append(map_and_join(nlp(doc)))
  processed_counter += 1
  if processed_counter % 100 == 0:
    print(processed_counter)
  # if processed_counter > 1000: break

# result = [map_and_join(nlp(doc)) for doc in filtered_dataset]


with open("/content/drive/MyDrive/diploma-llm/data/lda/dataset_lemmatized_full.json", "w") as outfile:
  outfile.write(json.dumps(result, indent=4))

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [48]:
result[45904]

'висновок час виконання дипломний роботи проаналізувати імпортний операція підприємство хімічний промисловість шлях підвищення ефективність розрахунок проводитися базі підприємство тов першому розділ описати теоретичний аспект імпорт поняття різновид імпортний діяльність особливість здійснення підприємство хімічний промисловість необхідний формула методи розрахунок ефективність другому розділ провести аналіз діяльність об’єкт дослідження тов зазначатися розкрити основний інформація діяльність найбільший хімічний концерн світі штаб квартира місія історія стратегічний напрямок розвиток напрямок діяльність головне особливість стало реалізація продукція група компанія напрямок хімія нафтохімічний продукт напівпродукт матеріал функціональний матеріал мономер рішення промисловість дисперсія смоли функціональний хімікат поверхнево активний сполука технологія каталізатор покриття продукт харчування догляд сировина косметика побутовий хімія харчовий інгредієнти фармацевтичний сировина здійснюва